In [1]:
import pandas as pd
import helium
import csv

In [2]:
_PATH_ = "https://public.minsky.cc/maria/market_channel_factors_db"

In [60]:
df = pd.read_csv(_PATH_ + "/" + "databases/03PreModelDatabaseOfWhitePotato.csv")

In [82]:
capitalOfRegions = pd.read_csv(_PATH_ + "/" + "databases/ProvinceCapitalOfRegions.csv")

In [4]:
df.columns

Index(['Unnamed: 0', 'NOMBREDD', 'NOMBREPV', 'NOMBREDI', 'CONGLOMERADO',
       'NSELUA', 'UA', 'crop', 'production', 'totalProduction',
       'percentProduction', 'soldProduction', 'totalSoldProduction',
       'percentSoldProduction', 'farmGatePricePerKg', 'farmGateIncomes',
       'totalFarmGateIncomes', 'percentFarmGateIncomes', 'plantedArea',
       'totalPlantedArea', 'percentPlantedArea', 'yield', 'volumeOfDistrict',
       'associative', 'informalPriceInformation', 'formalPriceInformation',
       'informalMetereologicalInformation', 'formalMetereologicalInformation',
       'localMarket', 'regionalMarket', 'limaMarket', 'agroindustryMarket',
       'exportMarket', 'MARKET_NAN', 'MARKETS', 'MARKETS_QTY', 'marketSize',
       'collector', 'wholesaler', 'retailer', 'enterprise', 'finalConsumer',
       'marketChannelDirectedness', 'pointsOfSale'],
      dtype='object')

In [52]:
def createNumberTimeVariable(df, variableName, newVariableName):
    df[newVariableName] = df[variableName].apply(lambda x: x.split(" ")).apply(lambda x: '{0:.3g}'.format(int(x[0]) + int(x[2])/60) if (len(x) > 2) else '{0:.2g}'.format(int(x[0])/60) if x[1] == "min" else int(x[0]))

# District time to Lima Market

In [7]:
places = df["NOMBREDI"] + ", " + df["NOMBREPV"] + ", " + df["NOMBREDD"]
places = places.unique()

distances_dtlm = {}
times_dtlm = {}

helium.start_chrome()
for place in places:
    fixed_place = place.lower().replace(" ", "+")
    url = f"http://google.com/maps/dir/{fixed_place}/Mercado+Productores,+Santa+Anita"
    helium.go_to(url)
    q = helium.S("div.section-directions-trip-numbers")
    box = helium.find_all(q)
    if len(box) == 0:
        distances_dtlm[place] = None
        times_dtlm[place] = None
        continue
    dist_and_time = box[0]
    dist_and_time = dist_and_time.web_element.text
    
    d_t = dist_and_time.split("\n")
    times_dtlm[place] = d_t[0]
    distances_dtlm[place] = d_t[1]

In [8]:
with open("04RoadTransportationDistrictToLimaMarket.csv", "w") as df:
    df.write("NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm\n")
    for key in distances_dtlm.keys():
        df.write("%s,%s,%s\n"%(key.replace(", ", ","), times_dtlm[key], distances_dtlm[key]))

In [53]:
districtToLimaMarket = pd.read_csv('04RoadTransportationDistrictToLimaMarket.csv')

In [54]:
createNumberTimeVariable(districtToLimaMarket, "DistanceInTime", "districtTimeToLimaMarket")

In [55]:
districtToLimaMarket

,NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm,districtTimeToLimaMarket
0,FLORIDA,BONGARA,AMAZONAS,20 h 53 min,1217 km,20.9
1,YAMBRASBAMBA,BONGARA,AMAZONAS,20 h 53 min,1228 km,20.9
2,CHILIQUIN,CHACHAPOYAS,AMAZONAS,22 h 38 min,1280 km,22.6
3,CHUQUIBAMBA,CHACHAPOYAS,AMAZONAS,19 h 47 min,1077 km,19.8
4,LA JALCA,CHACHAPOYAS,AMAZONAS,21 h 44 min,1137 km,21.7
...,...,...,...,...,...,...
381,TACNA,TACNA,TACNA,17 h 29 min,1221 km,17.5
382,HEROES ALBARRACIN,TARATA,TACNA,18 h 35 min,1283 km,18.6
383,SITAJARA,TARATA,TACNA,18 h 35 min,1283 km,18.6
384,TARATA,TARATA,TACNA,18 h 35 min,1283 km,18.6


In [56]:
districtToLimaMarket.to_csv('04RoadTransportationDistrictToLimaMarket.csv')


# District time to province capital of Region

In [62]:
def getCapitalProvinceDestination(region):
    row_selected = capitalOfRegions[capitalOfRegions["NOMBREDD"] == region].reset_index()
    return (row_selected["NOMBREPV"] +  ", " + row_selected["NOMBREDD"])[0]

In [69]:
region = "LA LIBERTAD"
getCapitalProvinceDestination(region)

'TRUJILLO, LA LIBERTAD'

In [70]:
places = df["NOMBREDI"] + ", " + df["NOMBREPV"] + ", " + df["NOMBREDD"]
places = places.unique()

distances_dtpc = {}
times_dtpc = {}

helium.start_chrome()
for place in places:
    region = place.split(", ")[-1]
    initial_place = place.lower().replace(" ", "+")
    destination_place = getCapitalProvinceDestination(region).lower().replace(" ", "+")
    url = f"http://google.com/maps/dir/{initial_place}/{destination_place}"
    helium.go_to(url)
    q = helium.S("div.section-directions-trip-numbers")
    box = helium.find_all(q)
    if len(box) == 0:
        distances_dtpc[place] = None
        times_dtpc[place] = None
        continue
    dist_and_time = box[0]
    dist_and_time = dist_and_time.web_element.text
    
    d_t = dist_and_time.split("\n")
    times_dtpc[place] = d_t[0]
    distances_dtpc[place] = d_t[1]


In [71]:
with open("04RoadTransportationDistrictToProvinceCapitalOfRegion.csv", "w") as df:
    df.write("NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm\n")
    for key in distances_dtpc.keys():
        df.write("%s,%s,%s\n"%(key.replace(", ", ","), times_dtpc[key], distances_dtpc[key]))

In [72]:
districtToProvinceCapitalOfRegion = pd.read_csv('04RoadTransportationDistrictToProvinceCapitalOfRegion.csv')

In [73]:
createNumberTimeVariable(districtToProvinceCapitalOfRegion, "DistanceInTime", "districtTimeToProvinceCapitalOfRegion")

In [74]:
districtToProvinceCapitalOfRegion

,NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm,districtTimeToProvinceCapitalOfRegion
0,FLORIDA,BONGARA,AMAZONAS,2 h 17 min,95.3 km,2.28
1,YAMBRASBAMBA,BONGARA,AMAZONAS,2 h 18 min,106 km,2.3
2,CHILIQUIN,CHACHAPOYAS,AMAZONAS,2 h 29 min,74.3 km,2.48
3,CHUQUIBAMBA,CHACHAPOYAS,AMAZONAS,6 h 23 min,232 km,6.38
4,LA JALCA,CHACHAPOYAS,AMAZONAS,1 h 50 min,71.2 km,1.83
...,...,...,...,...,...,...
381,TACNA,TACNA,TACNA,0 min,0 km,0
382,HEROES ALBARRACIN,TARATA,TACNA,1 h 40 min,88.4 km,1.67
383,SITAJARA,TARATA,TACNA,1 h 40 min,88.4 km,1.67
384,TARATA,TARATA,TACNA,1 h 40 min,88.4 km,1.67


In [75]:
districtToProvinceCapitalOfRegion.to_csv('04RoadTransportationDistrictToProvinceCapitalOfRegion.csv')

# Province capital time to Lima Market

In [84]:
places = capitalOfRegions["NOMBREPV"] + ", " + capitalOfRegions["NOMBREDD"]
places = places.unique()

distances_pctlm = {}
times_pctlm = {}

helium.start_chrome()
for place in places:
    region = place.split(", ")[-1]
    fixed_place = getCapitalProvinceDestination(region).lower().replace(" ", "+")
    url = f"http://google.com/maps/dir/{fixed_place}/Mercado+Productores,+Santa+Anita"
    helium.go_to(url)
    q = helium.S("div.section-directions-trip-numbers")
    box = helium.find_all(q)
    if len(box) == 0:
        distances_pctlm[place] = None
        times_pctlm[place] = None
        continue
    dist_and_time = box[0]
    dist_and_time = dist_and_time.web_element.text
    
    d_t = dist_and_time.split("\n")
    times_pctlm[place] = d_t[0]
    distances_pctlm[place] = d_t[1]
    


In [87]:
with open("04RoadTransportationProvinceCapitalToLimaMarket.csv", "w") as df:
    df.write("NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm\n")
    for key in distances_pctlm.keys():
        df.write("%s,%s,%s\n"%(key.replace(", ", ","), times_pctlm[key], distances_pctlm[key]))

In [88]:
provinceCapitalToLimaMarket = pd.read_csv('04RoadTransportationProvinceCapitalToLimaMarket.csv')

In [91]:
createNumberTimeVariable(provinceCapitalToLimaMarket, "DistanceInTime", "provinceCapitalToLimaMarket")

In [92]:
provinceCapitalToLimaMarket

,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm,provinceCapitalToLimaMarket
0,CHACHAPOYAS,AMAZONAS,20 h 20 min,1229 km,20.3
1,HUARAZ,ANCASH,6 h 46 min,411 km,6.77
2,ABANCAY,APURIMAC,16 h 28 min,938 km,16.5
3,AREQUIPA,AREQUIPA,14 h 56 min,1008 km,14.9
4,AYACUCHO,AYACUCHO,8 h 32 min,553 km,8.53
5,CAJAMARCA,CAJAMARCA,14 h 40 min,867 km,14.7
6,CUSCO,CUSCO,20 h 33 min,1132 km,20.6
7,HUANCAVELICA,HUANCAVELICA,6 h 1 min,420 km,6.02
8,HUANUCO,HUANUCO,8 h 12 min,380 km,8.2
9,ICA,ICA,3 h 48 min,297 km,3.8


In [94]:
provinceCapitalToLimaMarket.to_csv('04RoadTransportationProvinceCapitalToLimaMarket.csv')